In [94]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import words, cmudict
import string
import re

In [95]:
df = pd.read_excel('C:/Users/Dev/OneDrive/Desktop/Jupyter/Input.xlsx')
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [96]:
#Check if indexing works
print(df.iloc[3]['URL'])

https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/


In [97]:
output_df = pd.DataFrame()
output_df['URL_ID'] = df['URL_ID']
output_df['URL'] = df['URL']
length = len(df)
print(length)

114


In [102]:
for i in range(0,10):
    url = df.iloc[i]['URL']
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    text_content = soup.get_text()
    name = i+37
    print(url)
    with open(f'{name}.txt', 'w') as file:
        file.write(text_content)
        
        with open(f'{name}.txt', 'r') as file:
            content = file.read()

        # Read the words to delete from the word file
        with open('StopWords_Names.txt', 'r') as word_file:
            words_to_delete = [word.strip() for word in word_file.read().split()]

        with open('StopWords_Geographic.txt', 'r') as word_file:
            words_to_delete = [word.strip() for word in word_file.read().split()]

        with open('StopWords_Generic.txt', 'r') as word_file:
            words_to_delete = [word.strip() for word in word_file.read().split()]

        with open('StopWords_GenericLong.txt', 'r') as word_file:
            words_to_delete = [word.strip() for word in word_file.read().split()]

        with open('StopWords_Currencies.txt', 'r') as word_file:
            words_to_delete = [word.strip() for word in word_file.read().split()]

        with open('StopWords_Auditor.txt', 'r') as word_file:
            words_to_delete = [word.strip() for word in word_file.read().split()]

        with open('StopWords_DatesandNumbers.txt', 'r') as word_file:
            words_to_delete = [word.strip() for word in word_file.read().split()]

        # Delete the words from the content
        for word in words_to_delete:
            content = content.replace(word, '')

        # Open the text file in write mode
        with open(f'{name}.txt', 'w') as file:
            file.write(content)
        
        with open(f'{name}.txt', 'r') as file:
            content = file.read()

        tokens = word_tokenize(content)
        from nltk.tokenize import word_tokenize

        # Read positive words from a file
        positive_words = set()
        with open('positive-words.txt', 'r') as file:
            for line in file:
                positive_words.add(line.strip())

        # Sample tokenized data
        tokens = word_tokenize(content)

        # Segregate positive words
        positive_tokens = [token for token in tokens if token.lower() in positive_words]

        # Print positive words
        positive_score = len(positive_tokens)
        print(positive_score)

        negative_words = set()
        with open('negative-words.txt', 'r') as file:
            for line in file:
                negative_words.add(line.strip())

        negative_tokens = [token for token in tokens if token.lower() in negative_words]
        negative_score = len(negative_tokens)
        print(negative_score)
        
        if(positive_score and negative_score)==0:
            continue

        polarity_score = (positive_score-negative_score)/(positive_score+negative_score) + 0.000001
        #print(polarity_score)
        subj_score = (positive_score+negative_score)/(len(tokens))
        #print(subj_score)

        sentences = sent_tokenize(content)
        a = len(set(tokens))
        b = len(set(sentences))
        avg_sentence_length = a/b
        #print(avg_sentence_length)

        pronouncing_dict = cmudict.dict()

        # Function to count syllables in a word
        def count_syllables(word):
            # Check if the word is in the pronouncing dictionary
            if word.lower() in pronouncing_dict:
                # Get the pronunciation of the word
                pronunciation = pronouncing_dict[word.lower()][0]
                # Count the number of syllables by counting stress marks
                return len([ph for ph in pronunciation if ph[-1].isdigit()])
            else:
                # Return 0 if the word is not in the dictionary (unknown word)
                return 0

        # Identify words with two or more syllables
        complex_words = [token for token in tokens if count_syllables(token) >= 2]
        num_complex = len(complex_words)
        percent_complex = len(complex_words)/len(tokens)
        #print(percent_complex)

        fog_index = 0.4*(avg_sentence_length + percent_complex)
        #print(fog_index)

        avg_word_sentence = len(tokens)/len(sentences)
        #print(avg_word_sentence)

        word_count = len(tokens)
        #print(word_count)

        # cleaned_tokens = [token for token in tokens if token not in string.punctuation]
        # num_clean_tokens = len(cleaned_tokens)
        def count_syllable_2(word):
            # Remove trailing "es" and consider it as a single syllable
            if word.endswith(("es","ed","ad")):
                word = word[:-2]

            # Count vowels in the word
            num_vowels = sum(1 for char in word.lower() if char in 'aeiou')

            return num_vowels
        syllable_counts = [count_syllable_2(word) for word in tokens]
        syl_count_word = len(syllable_counts)
        avg_word_length = len(text_content)/len(tokens)
        
        pattern = r'\b(I|we|my|ours|us)\b'

        # Find and count personal pronouns
        matches = re.findall(pattern,text_content, flags=re.IGNORECASE)
        count_personal = len(matches)
        #print(count_personal)

        output_df.at[i, 'POSITIVE SCORE'] = positive_score
        output_df.at[i,'POSITIVE SCORE'] = positive_score
        output_df.at[i,'NEGATIVE SCORE'] = negative_score
        output_df.at[i,'POLARITY SCORE'] = polarity_score
        output_df.at[i,'SUBJECTIVITY SCORE'] = subj_score
        output_df.at[i,'AVERAGE SENTENCE LENGTH'] = avg_sentence_length
        output_df.at[i,'PERCENTAGE OF COMPLEX WORDS'] = percent_complex
        output_df.at[i,'FOG INDEX'] = fog_index
        output_df.at[i,'AVERAGE WORDS PER SENTENCE'] = avg_word_sentence
        output_df.at[i,'COMPLEX WORD COUNT'] = num_complex
        output_df.at[i,'WORD COUNT'] = word_count
        output_df.at[i,'SYLLABLE PER WORD'] = syl_count_word
        output_df.at[i,'PERSONAL PRONOUNS'] = count_personal
        output_df.at[i,'AVG WORD LENGTH'] = avg_word_length


https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
110
37
https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/
97
43
https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/
102
43
https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/
110
31
https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/
93
30
https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/
88
26
https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/
63
15
https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
0
0
https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/
74
16
https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/
110
48


In [103]:
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVERAGE SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVERAGE WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,110.0,37.0,0.496600,0.048403,13.372093,0.395456,5.507020,35.313953,1201.0,3037.0,3037.0,27.0,6.455384
1,38,https://insights.blackcoffer.com/what-if-the-c...,97.0,43.0,0.385715,0.052711,9.715789,0.312877,4.011466,27.957895,831.0,2656.0,2656.0,30.0,5.931852
2,39,https://insights.blackcoffer.com/what-jobs-wil...,102.0,43.0,0.406898,0.049320,11.072165,0.386735,4.583560,30.309278,1137.0,2940.0,2940.0,26.0,6.278571
3,40,https://insights.blackcoffer.com/will-machine-...,110.0,31.0,0.560285,0.049526,9.311321,0.340358,3.860672,26.607477,969.0,2847.0,2847.0,44.0,6.043203
4,41,https://insights.blackcoffer.com/will-ai-repla...,93.0,30.0,0.512196,0.041123,12.119565,0.345035,4.985840,32.510870,1032.0,2991.0,2991.0,45.0,6.088599


In [104]:
output_df.to_excel('output.xlsx', index=False)